# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [1]:
import findspark
findspark.init('/home/macaubas/spark-3.2.1-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("Basics").getOrCreate()

22/05/10 20:32:19 WARN Utils: Your hostname, macaubas-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/05/10 20:32:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 20:32:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/10 20:32:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/10 20:32:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


#### Imports 

In [117]:
# Tipos necessários
from pyspark.sql.types import (StructField, 
                               StringType, 
                               IntegerType, 
                               StructType,
                               DateType, 
                               FloatType)


# Funções necessárias
from pyspark.sql.functions import (dayofmonth, 
                                   dayofweek, 
                                   month,
                                   weekofyear,
                                   year,
                                   date_format,
                                   format_number,
                                   mean,
                                   countDistinct, 
                                   avg, 
                                   stddev,
                                   max,
                                   min,
                                   count,
                                   corr)



#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [11]:
#Criando Schema
data_schema = [StructField('Date', DateType(), True),
               StructField('Open', FloatType(), True),
               StructField('High', FloatType(), True),
               StructField('Low', FloatType(), True),
               StructField('Close', FloatType(), True),
               StructField('Volume', IntegerType(), True),
               StructField('Adj Close', FloatType(), True)]

final_struc = StructType(fields = data_schema)

# Carregando os dados
df = spark.read.csv('walmart_stock.csv', 
                    header = True, 
                    schema = final_struc)



#### What are the column names?

In [5]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [6]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [19]:
df.head(5)

[Row(Date=datetime.date(2012, 1, 3), Open=59.970001220703125, High=61.060001373291016, Low=59.869998931884766, Close=60.33000183105469, Volume=12668800, Adj Close=52.61923599243164),
 Row(Date=datetime.date(2012, 1, 4), Open=60.209999084472656, High=60.349998474121094, Low=59.470001220703125, Close=59.709999084472656, Volume=9593300, Adj Close=52.07847595214844),
 Row(Date=datetime.date(2012, 1, 5), Open=59.349998474121094, High=59.619998931884766, Low=58.369998931884766, Close=59.41999816894531, Volume=12768200, Adj Close=51.825538635253906),
 Row(Date=datetime.date(2012, 1, 6), Open=59.41999816894531, High=59.45000076293945, Low=58.869998931884766, Close=59.0, Volume=8069400, Adj Close=51.45922088623047),
 Row(Date=datetime.date(2012, 1, 9), Open=59.029998779296875, High=59.54999923706055, Low=58.91999816894531, Close=59.18000030517578, Volume=6679300, Adj Close=51.616214752197266)]

#### Use describe() to learn about the DataFrame.

In [23]:
df.describe().show()

+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|             Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|             1258|             1258|             1258|             1258|             1258|             1258|
|   mean|72.35785375452572| 72.8393880756178|71.91860094964979|72.38844997363553|8222093.481717011|67.23883840200064|
| stddev|6.768090251767697|6.768186825250206|6.744075739203606|6.756859160119612|  4519780.8431556|6.722609385249684|
|    min|            56.39|            57.06|             56.3|            56.42|          2094900|         50.36369|
|    max|             90.8|            90.97|            89.25|            90.47|         80898100|        84.914215|
+-------+-----------------+-----------------+-----------

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [77]:
summary = df.describe()

summary.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [80]:
summary.columns

['summary', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [82]:
# Ajustando Schema
for col in summary.columns:
    if col not in ["summary","Volume"]:
        summary = summary.withColumn(col, summary[col].cast("float"))
    elif col == "Volume":
        summary = summary.withColumn(col, summary[col].cast("int"))

summary.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: float (nullable = true)



In [83]:
summary.select(['summary', 
                format_number("Open", 2).alias("Open"),
                format_number("High", 2).alias("High"),
                format_number("Low", 2).alias("Low"),
                format_number("Close", 2).alias("Close"),
                format_number("Volume", 2).alias("Volume"),
                format_number("Adj Close", 2).alias("Adj Close"),]).show()

+-------+--------+--------+--------+--------+-------------+---------+
|summary|    Open|    High|     Low|   Close|       Volume|Adj Close|
+-------+--------+--------+--------+--------+-------------+---------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00| 1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.00|    67.24|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,780.00|     6.72|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|    50.36|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,100.00|    84.91|
+-------+--------+--------+--------+--------+-------------+---------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [85]:
new_df = df.withColumn('HV Ratio', df['High']/df['Volume']).select('HV Ratio')

new_df.show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714682786927E-6|
|6.290848662516662E-6|
| 4.66941298944916E-6|
| 7.36733843444859E-6|
|8.915604814435727E-6|
|8.644477449144044E-6|
|9.351828386844425E-6|
| 8.29141562102703E-6|
|7.712212051589609E-6|
|7.071764777688419...|
|1.015495462653464...|
|  6.5763540967921E-6|
| 5.90145296180676E-6|
|8.547679390846264E-6|
|8.420709512685392E-6|
|1.041448335142357...|
|8.316075435382035E-6|
|9.721183804158345E-6|
|8.029435987746889E-6|
|6.307432228123159E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [95]:
highest_price = df.select(max(df['High'])).collect()[0][0]

df.filter(df['High'] == highest_price).collect()[0][0]

datetime.date(2015, 1, 13)

#### What is the mean of the Close column?

In [97]:
df.agg({"Close":"mean"}).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844997363553|
+-----------------+



#### What is the max and min of the Volume column?

In [100]:
df.agg(max(df['Volume']), min(df['Volume'])).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [107]:
df.filter(df['Close'] < 60).agg(count(df['Close'])).collect()[0][0]

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [116]:
grt_80 = df.filter(df['High'] > 80).agg(count(df['High'])).collect()[0][0]
total = df.agg(count(df['High'])).collect()[0][0]

grt_80*100/total

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [119]:
df.agg(corr(df['High'], df['Volume'])).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061799556|
+-------------------+



#### What is the max High per year?

In [124]:
df = df.withColumn('Year', year(df['Date']))

df.groupBy('Year').agg(max(df['High']).alias('Max')).orderBy('Year').show()

+----+-----+
|Year|  Max|
+----+-----+
|2012| 77.6|
|2013|81.37|
|2014|88.09|
|2015|90.97|
|2016|75.19|
+----+-----+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [127]:
df = df.withColumn('Month', month(df['Date']))

df.groupBy('Month').agg(mean(df['Close']).alias('avg')).orderBy('Month').show()

+-----+-----------------+
|Month|              avg|
+-----+-----------------+
|    1| 71.4480196131338|
|    2|71.30680438169499|
|    3|71.77794376266337|
|    4|72.97361900692894|
|    5|72.30971685445533|
|    6| 72.4953774506191|
|    7|74.43971944078106|
|    8| 73.0298185521906|
|    9|72.18411782208611|
|   10| 71.5785454489968|
|   11|72.11108927207418|
|   12|72.84792482628012|
+-----+-----------------+



# Great Job!